In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,46208
2,Huelva,Confirmados PDIA 14 días,362
3,Huelva,Tasa PDIA 14 días,"70,54192567765068"
4,Huelva,Confirmados PDIA 7 días,176
5,Huelva,Tasa PDIA 7 dias,"34,29662684880254"
6,Huelva,Total Confirmados,46412
7,Huelva,Curados,45154
8,Huelva,Fallecidos,420


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  46208.0


/tmp/ipykernel_72266/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12700.0


/tmp/ipykernel_72266/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_72266/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_72266/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_72266/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 3127 personas en los últimos 7 días 

Un positivo PCR cada 1712 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,46208.0,176.0,362.0,513170.0,34.296627,70.541926,26.0
Huelva-Costa,27334.0,106.0,213.0,289548.0,36.608783,73.562933,15.0
Huelva (capital),12700.0,46.0,84.0,143837.0,31.980645,58.399438,10.0
Condado-Campiña,14353.0,54.0,104.0,156231.0,34.564203,66.568095,9.0
Almonte,2329.0,10.0,19.0,24507.0,40.804668,77.528869,3.0
Palos de la Frontera,1073.0,8.0,21.0,11742.0,68.131494,178.845171,2.0
Aljaraque,1643.0,10.0,10.0,21474.0,46.567943,46.567943,1.0
Ayamonte,2111.0,6.0,17.0,21104.0,28.430629,80.553450,1.0
Bonares,467.0,2.0,3.0,6060.0,33.003300,49.504950,1.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Campofrío,52.0,7.0,25.0,713.0,981.767181,3506.311360,0.0
Granado (El),46.0,1.0,6.0,532.0,187.969925,1127.819549,0.0
Cartaya,2127.0,30.0,59.0,20083.0,149.380073,293.780810,1.0
San Bartolomé de la Torre,386.0,6.0,8.0,3761.0,159.532039,212.709386,1.0
Minas de Riotinto,187.0,4.0,7.0,3812.0,104.931794,183.630640,1.0
Palos de la Frontera,1073.0,8.0,21.0,11742.0,68.131494,178.845171,2.0
Cabezas Rubias,53.0,1.0,1.0,706.0,141.643059,141.643059,0.0
Moguer,2023.0,11.0,24.0,21867.0,50.304111,109.754424,0.0
Santa Bárbara de Casa,47.0,1.0,1.0,1043.0,95.877277,95.877277,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Granado (El),46.0,1.0,6.0,532.0,187.969925,1127.819549,0.0,0.166667
Bollullos Par del Condado,1298.0,1.0,4.0,14387.0,6.950719,27.802878,0.0,0.250000
Lepe,3047.0,4.0,15.0,27880.0,14.347202,53.802009,0.0,0.266667
Campofrío,52.0,7.0,25.0,713.0,981.767181,3506.311360,0.0,0.280000
Sierra de Huelva-Andévalo Central,4093.0,14.0,43.0,67391.0,20.774287,63.806740,1.0,0.325581
Nerva,254.0,1.0,3.0,5169.0,19.346102,58.038305,0.0,0.333333
Isla Cristina,3162.0,6.0,17.0,21393.0,28.046557,79.465246,1.0,0.352941
Ayamonte,2111.0,6.0,17.0,21104.0,28.430629,80.553450,1.0,0.352941
Palos de la Frontera,1073.0,8.0,21.0,11742.0,68.131494,178.845171,2.0,0.380952
